In [1]:
import pandas as pd
import numpy as np

pairs = pd.read_csv('HuRI.tsv', sep='\t', header=None,names=['protein1', 'protein2'])
pairs

,protein1,protein2
0,ENSG00000000005,ENSG00000061656
1,ENSG00000000005,ENSG00000099968
2,ENSG00000000005,ENSG00000104765
3,ENSG00000000005,ENSG00000105383
4,ENSG00000000005,ENSG00000114455
...,...,...
52543,ENSG00000273899,ENSG00000273899
52544,ENSG00000275302,ENSG00000278619
52545,ENSG00000275774,ENSG00000275774
52546,ENSG00000276070,ENSG00000278619


In [2]:
prots = set(pairs['protein1']).union(set(pairs['protein2']))
prots = np.array(list(prots))
len(prots)

8272

In [3]:
embedtestCSV = pd.read_csv('embeddings_test.csv',index_col=0)
embeddings_test_array = np.array(list(embedtestCSV.values))
test_embeddings = {embedtestCSV.index[i]: embeddings_test_array[i] for i in range(len(embeddings_test_array))}

In [7]:
prots_checked = []
for prot in prots:
    prot1 = 'http:/www.ensembl.org/Homo_sapiens/Location/View?r=' + prot
    if prot1 in test_embeddings:
        prots_checked.append(prot)

In [8]:
len(prots_checked)

8099

In [9]:
np.savetxt('HuRI_proteins.txt', prots_checked, fmt='%s', delimiter='\n')

# Generate negatives

In [22]:
new_pairs = []
for ps in pairs.values:
    if ps[0] in prots_checked and ps[1] in prots_checked:
        new_pairs.append((ps[0],ps[1]))

setsize = len(new_pairs)

In [20]:
set_pairs = set(new_pairs)
set_pairs

{('ENSG00000117899', 'ENSG00000120289'),
 ('ENSG00000188610', 'ENSG00000203786'),
 ('ENSG00000036565', 'ENSG00000132406'),
 ('ENSG00000082781', 'ENSG00000188581'),
 ('ENSG00000115295', 'ENSG00000176049'),
 ('ENSG00000107130', 'ENSG00000177427'),
 ('ENSG00000154639', 'ENSG00000172005'),
 ('ENSG00000186453', 'ENSG00000189252'),
 ('ENSG00000239886', 'ENSG00000264343'),
 ('ENSG00000105072', 'ENSG00000135249'),
 ('ENSG00000162366', 'ENSG00000181704'),
 ('ENSG00000100652', 'ENSG00000127564'),
 ('ENSG00000137393', 'ENSG00000152977'),
 ('ENSG00000143947', 'ENSG00000183283'),
 ('ENSG00000196843', 'ENSG00000244362'),
 ('ENSG00000180096', 'ENSG00000180096'),
 ('ENSG00000106049', 'ENSG00000186501'),
 ('ENSG00000061337', 'ENSG00000129083'),
 ('ENSG00000115194', 'ENSG00000118402'),
 ('ENSG00000105072', 'ENSG00000132938'),
 ('ENSG00000101856', 'ENSG00000144021'),
 ('ENSG00000154127', 'ENSG00000163743'),
 ('ENSG00000153179', 'ENSG00000189067'),
 ('ENSG00000100146', 'ENSG00000183283'),
 ('ENSG000001871

In [21]:
negatives = set()

for p1 in prots_checked:
    for p2 in prots_checked:
        if (p1, p2) not in set_pairs and (p2, p1) not in set_pairs and p1 != p2 and (p2, p1) not in negatives and (p1, p2) not in negatives:
            negatives.add((p1, p2))

In [23]:
negatives_df = pd.DataFrame(list(negatives), columns=['protein1', 'protein2'])
negatives_df = negatives_df.sample(n=setsize, random_state=42)
negatives_df.to_csv('HuRI_negatives.tsv', sep='\t', index=False, header=False)
